In [1]:
import time
import sys
from pymata4 import pymata4
import threading

In [2]:
LED_PIN = 4
Button_pin=2
timerTime = 5 # time in sec

In [3]:

board = pymata4.Pymata4(com_port = "COM4")


pymata4:  Version 1.15

Copyright (c) 2020 Alan Yorinks All Rights Reserved.

Opening COM4...

Waiting 4 seconds(arduino_wait) for Arduino devices to reset...
Arduino compatible device found and connected to COM4

Retrieving Arduino Firmware ID...
Arduino Firmware ID: 1.2 FirmataExpress.ino

Retrieving analog map...
Auto-discovery complete. Found 20 Digital Pins and 6 Analog Pins




In [15]:
def LED_ON():
    while True:
        button_state = board.digital_read(Button_pin)
        if button_state == 0:  # Button is pressed
            print("Button pressed!")
            board.digital_write(LED_PIN, 1)  # Turn LED on
            time.sleep(timerTime)  # Wait for TIMER_TIME seconds
            board.digital_write(LED_PIN, 0)  # Turn LED off
        time.sleep(0.1)  # Short delay to avoid busy waiting


In [16]:
board.set_pin_mode_digital_output(LED_PIN)
board.set_pin_mode_digital_input_pullup(Button_pin,callback=LED_ON)

In [17]:
threading.Thread(target=LED_ON, daemon=True).start()

In [18]:
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Program stopped.")



Program stopped.


h
